In [13]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [38]:
#Parameters
samplenum = 50
epochs = 100
hiddenlayers = [90]
input_size = 3
output_size = 3
learning_rate = 0.001
time_length = 60; #seconds

In [39]:
# Generate simulation
dyn = d.PyDyn('Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [40]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [41]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        #print(f'shape of dy/dp: {dy_dp.shape}')
        #print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        return grad_input

Simulate = Simulate.apply

## Building the Model

In [42]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [45]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()
weight_c1 = 1 # p start condition
weight_c2 = 1 # y start condition
losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        p_pred = model(y_truth)
        y_pred = Simulate(p_pred)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        smoothness_error = criterion(p_pred[0:3*(time_length-1)], p_pred[3:3*time_length])
        #loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        #loss = criterion(y_pred.float(), y_truth) + weight_c1*(sum(p_pred[0:3]-torch.tensor(dyn.p_init[0:3])))**2  # MSE + start condition penalty + p smoothness condition penalty
        loss = criterion(y_pred.float(), y_truth) + weight_c1*criterion(p_pred[0:3].double(), torch.tensor(dyn.p_init[0:3])) + weight_c2*smoothness_error# MSE + start condition penalty + p smoothness condition penalty
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

epoch:   0/100  loss: 2.27975308
epoch:   1/100  loss: 1.98562084
epoch:   2/100  loss: 1.53118231
epoch:   3/100  loss: 0.73342367
epoch:   4/100  loss: 0.48224390
epoch:   5/100  loss: 0.39723121
epoch:   6/100  loss: 0.39132231
epoch:   7/100  loss: 0.38500593
epoch:   8/100  loss: 0.37829823
epoch:   9/100  loss: 0.37139875
epoch:  10/100  loss: 0.36428706
epoch:  11/100  loss: 0.35697664
epoch:  12/100  loss: 0.34947179
epoch:  13/100  loss: 0.34172603
epoch:  14/100  loss: 0.33357242
epoch:  15/100  loss: 0.30452962
epoch:  16/100  loss: 0.14911450
epoch:  17/100  loss: 0.01814017
epoch:  18/100  loss: 0.00373714
epoch:  19/100  loss: 0.00085121
epoch:  20/100  loss: 0.00039413
epoch:  21/100  loss: 0.00073770
epoch:  22/100  loss: 0.00283775
epoch:  23/100  loss: 0.00163454
epoch:  24/100  loss: 0.00212314
epoch:  25/100  loss: 0.00104888
epoch:  26/100  loss: 0.00087157
epoch:  27/100  loss: 0.00082102
epoch:  28/100  loss: 0.00083013
epoch:  29/100  loss: 0.00093871
epoch:  30

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [53]:
y_target= torch.tensor([0, 2, 0.5])
p = model(y_target)
p = p.detach().numpy()

y_pred_state = dyn.compute(p)
y_pred = y_pred_state.y[-3:]

print(y_pred)
print(y_target)
print(f'p trajecory: {p}')
print(f"y trajectory: {y_pred_state.y}")
error = 0
for i in range(time_length-1):
    step = sum((p[3*i:3*i+3] - p[3*i+3:3*i+6])**2)
    error = error + step
print(error/time_length)


[0.00401509 1.98388322 0.48141816]
tensor([0.0000, 2.0000, 0.5000])
p trajecory: [-4.21194732e-03  2.99492264e+00 -1.17538869e-03  1.41971314e+00
 -3.83457631e-01  5.21476090e-01 -3.77948165e-01  5.39184034e-01
  6.50334239e-01 -8.88038278e-01  6.70214534e-01 -6.37364745e-01
  6.04890585e-01 -6.59534156e-01 -1.19655691e-01  9.54622030e-01
  7.08152413e-01 -7.03089595e-01 -1.87960416e-01  5.71996160e-02
  1.26621199e+00 -5.28571248e-01 -7.13740051e-01 -9.47222352e-01
 -2.33718544e-01  8.46694931e-02 -1.43422306e-01 -7.10699737e-01
 -7.43213415e-01  3.23251545e-01 -5.05875647e-02  1.52284637e-01
  1.48828757e+00  1.09069109e+00 -1.09838033e+00 -2.23105535e-01
  6.53168917e-01  5.32971740e-01 -2.53608555e-01  2.20701218e-01
  1.01644725e-01  4.31829989e-01  8.27530384e-01  1.10764563e+00
 -9.38422978e-01 -1.37930006e-01  6.48558974e-01  3.64954211e-02
 -8.99044335e-01 -4.41821814e-01 -1.04966390e+00 -9.82296407e-01
 -9.26035643e-01  1.53821409e+00  9.41120625e-01  1.19896913e+00
 -7.49491

In [51]:
p[3*i+3:3*i+6]

array([], dtype=float32)

## Torch Script Conversion and Saving

In [36]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)
original = model(test_input)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-12:])
print(original)

tensor([1.9574, 0.0000, 0.0000, 2.7701, 0.0000], grad_fn=<SliceBackward>)


In [ ]:
# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")